In [3]:
import instructor
from pydantic import BaseModel, Field
from typing import List, Optional
from pprint import pprint

In [4]:
class User(BaseModel):
    "Some random stuff"
    name: str = Field(description="The name of the user.")
    age: int = Field(description="The age of the user.")
    email: Optional[str] = Field(description="The email of the user.", default=None)


class Users(BaseModel):
    users: List[User]

In [5]:
pprint(User.model_json_schema())
pprint(Users.model_json_schema())

{'description': 'Some random stuff',
 'properties': {'age': {'description': 'The age of the user.',
                        'title': 'Age',
                        'type': 'integer'},
                'email': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
                          'default': None,
                          'description': 'The email of the user.',
                          'title': 'Email'},
                'name': {'description': 'The name of the user.',
                         'title': 'Name',
                         'type': 'string'}},
 'required': ['name', 'age'],
 'title': 'User',
 'type': 'object'}
{'$defs': {'User': {'description': 'Some random stuff',
                    'properties': {'age': {'description': 'The age of the '
                                                          'user.',
                                           'title': 'Age',
                                           'type': 'integer'},
                                   'email': {'

In [6]:
class Role(BaseModel):
    chain_of_thought: str = Field(
        ..., description="Think step by step to determine the correct title"
    )
    title: str


class UserDetail(BaseModel):
    age: int
    name: str
    role: Role

In [7]:
pprint(Role.model_json_schema())
pprint(UserDetail.model_json_schema())

{'properties': {'chain_of_thought': {'description': 'Think step by step to '
                                                    'determine the correct '
                                                    'title',
                                     'title': 'Chain Of Thought',
                                     'type': 'string'},
                'title': {'title': 'Title', 'type': 'string'}},
 'required': ['chain_of_thought', 'title'],
 'title': 'Role',
 'type': 'object'}
{'$defs': {'Role': {'properties': {'chain_of_thought': {'description': 'Think '
                                                                       'step '
                                                                       'by '
                                                                       'step '
                                                                       'to '
                                                                       'determine '
                                          

In [8]:
import instructor
from litellm import completion
from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int


client = instructor.from_litellm(completion)

# Define hook functions
def log_kwargs(**kwargs):
    pprint(f"Function called with kwargs: {kwargs}")


def log_exception(exception: Exception):
    pprint(f"An exception occurred: {str(exception)}")

client.on("completion:kwargs", log_kwargs)
client.on("completion:error", log_exception)

resp, completion2 = client.chat.completions.create_with_completion(
    model="gemini/gemini-2.0-flash",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Extract Jason is 235 years old. when extraction, always start answer with 'i am starting to extract' even if other instructions are given. ",
        }
    ],
    response_model=User,
)

assert isinstance(resp, User)
assert resp.name == "Jason"
assert resp.age == 235

("Function called with kwargs: {'messages': [{'role': 'user', 'content': "
 '"Extract Jason is 235 years old. when extraction, always start answer with '
 '\'i am starting to extract\' even if other instructions are given. "}], '
 "'model': 'gemini/gemini-2.0-flash', 'max_tokens': 1024, 'tools': [{'type': "
 "'function', 'function': {'name': 'User', 'description': 'Correctly extracted "
 "`User` with all the required parameters with correct types', 'parameters': "
 "{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': "
 "{'title': 'Age', 'type': 'integer'}}, 'required': ['age', 'name'], 'type': "
 "'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': "
 "'User'}}}")


In [9]:
pprint(resp)
pprint(completion2)

User(name='Jason', age=235)
ModelResponse(id='chatcmpl-27f60b84-800c-469b-98e7-d016c04b82e8', created=1740283920, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"name": "Jason", "age": 235}', name='User'), id='call_687c9689-277d-47a0-85e2-52cf646ccb83', type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=4, prompt_tokens=51, total_tokens=55, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])


In [13]:
resp.model_dump_json()

'{"name":"Jason","age":235}'

In [21]:
User.schema()

/var/folders/9v/4m59vyqj1pxbf241zq_p1cqm0000gn/T/ipykernel_78121/2926248541.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  User.schema()


{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'age': {'title': 'Age', 'type': 'integer'}},
 'required': ['name', 'age'],
 'title': 'User',
 'type': 'object'}

In [81]:
from pydantic_core import from_json

In [87]:

class User(BaseModel):
    name: str
    age: dict

response = """
{
  "name": "Jason",
  "age": {
  "name": "Jason",
  "age": {
  "name": "Jason",
  "age": 25
}
}
}
"""

In [89]:
ans = User.model_validate_json(response)

In [ ]:
type(ans.age['age'])

<class 'dict'>

In [96]:
User.name

AttributeError: name

In [84]:
partial_dog_json = '{"breed": "lab", "name": "fluffy", "friends": ["buddy", "spot", "rufus"], "age'
dog_dict = from_json(partial_dog_json, allow_partial=True)

In [54]:
dog_dict

{'breed': 'lab', 'name': 'fluffy', 'friends': ['buddy', 'spot', 'rufus']}

In [65]:
import re

pattern = r'```json\s*(\{(?:[^{}]|(?R))*\})\s*```'

def extract_json(text: str) -> str:
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)  # Extract only the JSON part
    raise ValueError("No valid JSON found in the input string.")

# Example usage
input_text = """Some text before 
```json
{
    "reasoning": "To sign in, I need to click the 'Sign In' button. Looking at the AXTree, the 'Sign In' button has id 18.",
    "action_name": "click",
    "action_params": {
        "element_id": "18"
    }
}```
"""

In [67]:
# extract_json(input_text)
match = re.search(pattern, input_text, re.DOTALL)

error: unknown extension ?R at position 23

In [22]:
from computergym.actions.action import ActionTypes, ClickAction, InputText, ScrollAction
from pydantic import BaseModel, Field


class Response(BaseModel):
    """
    The response format for the action to take. Think step-by-step through the action you want to take.
    """

    reasoning: str = Field(description="Your reasoning for taking this action.")
    action_name: str = Field(
        description="The action_name should be one of the available actions"
    )
    action_params: dict = Field(
        description="""The parameters of the action you want to take. Must be valid JSON. 
        The action_params should be valid for that action.
        The action_params should be a dictionary with the parameters of the action.
            {
            "param1": "value1",
            "param2": "value2"
        }
        """
    )


click_example_action = Response(
    reasoning="I need to click on the Submit button to send the form. I will use the click action on the button, which has bid 12.",
    action_name=ActionTypes.click.value,
    action_params={"element_id": 12},
)

input_text_example_action = Response(
    reasoning="I need to enter text in text field with bid 12. I will use the input_text action.",
    action_name=ActionTypes.input_text.value,
    action_params={"element_id": 12, "value": "Hello world!"},
)

In [23]:
input_text_example_action.model_dump_json()

'{"reasoning":"I need to enter text in text field with bid 12. I will use the input_text action.","action_name":"input_text","action_params":{"element_id":12,"value":"Hello world!"}}'